In [8]:
import asyncio
from pyzeebe import ZeebeWorker, Job, create_camunda_cloud_channel, ZeebeClient

In [9]:
ClusterURL = "fc43de37-01a5-41a0-a9d8-51d621169f35.dsm-1.zeebe.camunda.io:443"
ClientID = "UPPG5KY-b6ZRN1Yf_iuXG87hucOa286k"
ClientSecret = "J~pZPr6LZ6NxBa2Qwr7DVd8YwZahW35v8RIAMsi~h~VThu0.76GzL73sLB37CR2r"
ClusterID = "fc43de37-01a5-41a0-a9d8-51d621169f35"
Region = "dsm-1"

In [10]:
channel = create_camunda_cloud_channel(client_id=ClientID, 
                                       client_secret=ClientSecret, 
                                       cluster_id=ClusterID, 
                                       region=Region)

In [11]:
# Create a zeebe client and worker
client = ZeebeClient(channel)
worker = ZeebeWorker(channel)

In [12]:
process_key = await client.run_process(bpmn_process_id='Kunde01', variables={})

In [13]:
@worker.task(task_type="BestellungVersendenType")
async def BestellungVersenden(Mail: str, Land: str, Anzahl: float):

    corr_key_a="Message_22fjeih"
    await client.publish_message(name=corr_key_a, correlation_key=corr_key_a, variables={"Mail": Mail,
        "Land": Land, "Anzahl": Anzahl})
    
    return {}

@worker.task(task_type="BestellungWeitergebenType")
async def BestellungWeitergeben(Mail: str, Land: str, Anzahl: float):

    corr_key_b="Message_1qhiank"
    await client.publish_message(name=corr_key_b, correlation_key=corr_key_b, variables={"Mail": Mail,
        "Land": Land, "Anzahl": Anzahl})
    
    return {}

@worker.task(task_type="FehlgeschlagenType")
async def Fehlgeschlagen(Mail: str, Land: str, Anzahl: float):

    corr_key_c="Message_2opgcoj"
    await client.publish_message(name=corr_key_c, correlation_key=corr_key_c, variables={"Message": "Erstellung fehlgeschlagen!"})
    
    return {}

@worker.task(task_type="AngebotAbgebenType")
async def AngebotAbgeben(Kosten: float):

    corr_key_d="Message_0hj1318"
    await client.publish_message(name=corr_key_d, correlation_key=corr_key_d, variables={"Kosten": Kosten})
    
    return {}

await worker.work()